### Using Scispacy

In [1]:
%pip install https://s3-us-west-2.amazonaws.com/ai2-s2-scispacy/releases/v0.4.0/en_core_sci_sm-0.4.0.tar.gz

     |████████████████████████████████| 15.6 MB 1.4 MB/s eta 0:00:01    |▍                               | 184 kB 253 kB/s eta 0:01:01
  Created wheel for en-core-sci-sm: filename=en_core_sci_sm-0.4.0-py3-none-any.whl size=15660350 sha256=df20d4bc86d2fea76cb01904d9f0acb1fcb3461a8f9583a77c7e0ccc8f526941
  Stored in directory: /Users/dragonzord/Library/Caches/pip/wheels/f0/79/11/5b19b0ba4b8fbe52561d7f2f5addbc97c5e20b417f573ce5f0
Successfully built en-core-sci-sm
Note: you may need to restart the kernel to use updated packages.


In [4]:
patient_notes = pd.read_csv("patient_notes.csv")
patient_notes["pn_history"][patient_notes["case_num"] == 0][:10]

NameError: name 'pd' is not defined

In [ ]:
case_num_zero = patient_notes["pn_history"][patient_notes["case_num"] == 0][:1500]
whole_case_num_zero = "\n".join(case_num_zero.to_list())
whole_case_num_zero[:1000]

In [ ]:
import prepare_jag
clean_note = prepare_jag.basic_clean2(whole_case_num_zero)
clean_note = prepare_jag.remove_stopwords(clean_note, exclude_words=["no"])
len(clean_note)

In [2]:
print("works1")
import scispacy
import spacy
print("works2")

#from scispacy.linking import EntityLinker

print("works3")
sci_nlp = spacy.load("en_core_sci_lg")
# This line takes a while, because we have to download ~1GB of data
# and load a large JSON file (the knowledge base). Be patient!
# Thankfully it should be faster after the first time you use it, because
# the downloads are cached.
print("works4")

works1
works2
works3
works4


In [1]:
# NOTE: The resolve_abbreviations parameter is optional, and requires that
# the AbbreviationDetector pipe has already been added to the pipeline. Adding
# the AbbreviationDetector pipe and setting resolve_abbreviations to True means
# that linking will only be performed on the long form of abbreviations.
sci_nlp.add_pipe("scispacy_linker", config={"linker_name": "umls"})

doc = sci_nlp(clean_note)
# Let's look at a random entity!
# doc comes from above
entity = doc.ents[1]
print(f"Name: {entity}")

# Each entity is linked to UMLS with a score
# (currently just char-3gram matching).
linker = nlp.get_pipe("scispacy_linker")
for umls_ent in entity._.kb_ents:
	print(linker.kb.cui_to_entity[umls_ent[0]])

NameError: name 'sci_nlp' is not defined